In [1]:
from utils import TokenQuestion, Word2VecSolver, RetrievalSolver, Question, Solver
import collections
import jieba
import pandas as pd
import numpy as np
import subprocess
from subprocess import PIPE
import shlex
import re
import csv
from time import time
ANS = ['a', 'b', 'c', 'd', 'e']
WINDOW = 10
VEC_SIZE = 100


In [2]:
jieba.set_dictionary('data/DICT_CK+jieba_lower')
jieba.load_userdict('data/zhwiki-zh-clean')
jieba.add_word('龐燮傍謝', freq=10, tag='xx')


Building prefix dict from /Users/tripper/Desktop/project/pixnet-nlp/data/DICT_CK+jieba_lower ...
Loading model from cache /var/folders/4_/b_9vfbwx41g4t48jrzr0r4vm0000gn/T/jieba.uaab2b61a375629a86d8bcbcdd9e861b1.cache
Loading model cost 1.179 seconds.
Prefix dict has been built succesfully.


In [12]:
grep = RetrievalSolver('data/content-zh-lower.txt')

In [3]:
w2v_jieba_cbow = Word2VecSolver('w2v-experiment/model/', 'cbow')

In [2]:
w2v_uni_cbow = Word2VecSolver('w2v-experiment/model/', 'uni-cbow')

In [3]:
val_set = pd.read_csv('question_sample/pre/raw_samples.csv')
test_set = pd.read_csv('question_sample/official/hackathon_1000.tsv', sep='\t', names=['no','content', 'ans', 'a', 'b', 'c', 'd', 'e', 'ans_ref', 'url', 'level'], quoting=csv.QUOTE_NONE)

In [7]:
# direct word2vec(unigram+cbow)
t = time()
correct, total = 0, 0
for index, row in val_set.iterrows():
    q = TokenQuestion(row, 'uni')
    w2v_uni_cbow.solve(q, uni_gram = True)
    if w2v_uni_cbow.prediction != Solver.UNSOLVED:
        if w2v_uni_cbow.prediction == q.ans: correct += 1
        total += 1
    
print('validation accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total validation samples: {}'.format(total))

temp = total
correct, total = 0, 0
for index, row in test_set.iterrows():
    q = TokenQuestion(row, 'uni')
    w2v_uni_cbow.solve(q, uni_gram = True)
    if w2v_uni_cbow.prediction != Solver.UNSOLVED:
        if w2v_uni_cbow.prediction == q.ans: correct += 1
        total += 1

elapsed = time() - t

print('test accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total test samples: {}'.format(total))
print('elapsed time: {:.2f} sec'.format(elapsed))
print('rate: {:.2f} sec/sample'.format(float(elapsed) / float(temp + total)))

# validation accuracy: 63.19%
# total validation samples: 546
# test accuracy: 59.60%
# total test samples: 1000
# elapsed time: 2.32 sec
# rate: 0.00 sec/sample

validation accuracy: 63.19%
total validation samples: 546
test accuracy: 59.60%
total test samples: 1000
elapsed time: 2.32 sec
rate: 0.00 sec/sample


In [15]:
# grep + word2vec(jieba+cbow) aggregation
t = time()
correct, total = 0, 0
grep_cached_pred = pd.read_csv('cached/grep_validation.csv')
grep_cached_pred.fillna('', inplace=True)
for (i1, r1), (i2, r2) in zip(val_set.iterrows(), grep_cached_pred.iterrows()):
    q = Question(r1)
    if r2.prediction != str(Solver.UNSOLVED):
        if r2.prediction == q.ans: correct += 1
        elif len(r2.key) > 0:
            w2v_jieba_cbow.solve(q, key = r2.key)
            if w2v_jieba_cbow.prediction == q.ans: correct += 1
        total += 1
print('validation accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total validation samples: {}'.format(total))

temp = total
correct, total = 0, 0
grep_cached_pred = pd.read_csv('cached/grep_test.csv')
grep_cached_pred.fillna('', inplace=True)
for (i1, r1), (i2, r2) in zip(test_set.iterrows(), grep_cached_pred.iterrows()):
    q = Question(r1)
    if r2.prediction != str(Solver.UNSOLVED):
        if r2.prediction == q.ans: correct += 1
        elif len(r2.key) > 0:
            w2v_jieba_cbow.solve(q, key = r2.key)
            if w2v_jieba_cbow.prediction == q.ans: correct += 1
        total += 1
elapsed = time() - t

print('test accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total test samples: {}'.format(total))
print('elapsed time: {:.2f} sec'.format(elapsed))
print('rate: {:.2f} sec/sample'.format(float(elapsed) / float(temp + total)))

# validation accuracy: 93.77%
# total validation samples: 546
# test accuracy: 86.80%
# total test samples: 1000
# elapsed time: 561.27 sec
# rate: 0.36 sec/sample

validation accuracy: 93.77%
total validation samples: 546
test accuracy: 86.80%
total test samples: 1000
elapsed time: 1.25 sec
rate: 0.00 sec/sample


In [8]:
# direct word2vec(jieba+cbow)
t = time()
correct, total = 0, 0
for index, row in val_set.iterrows():
    q = TokenQuestion(row, 'jieba')
    w2v_jieba_cbow.solve(q)
    if w2v_jieba_cbow.prediction != Solver.UNSOLVED:
        if w2v_jieba_cbow.prediction == q.ans: correct += 1
        total += 1
print('validation accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total validation samples: {}'.format(total))

temp = total
correct, total = 0, 0
for index, row in test_set.iterrows():
    q = TokenQuestion(row, 'jieba')
    w2v_jieba_cbow.solve(q)
    if w2v_jieba_cbow.prediction != Solver.UNSOLVED:
        if w2v_jieba_cbow.prediction == q.ans: correct += 1
        total += 1

elapsed = time() - t

print('test accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total test samples: {}'.format(total))
print('elapsed time: {:.2f} sec'.format(elapsed))
print('rate: {:.2f} sec/sample'.format(float(elapsed) / float(temp + total)))

# validation accuracy: 79.85%
# total validation samples: 546
# test accuracy: 79.70%
# total test samples: 1000
# elapsed time: 2.48 sec
# rate: 0.00 sec/sample

validation accuracy: 79.85%
total validation samples: 546
test accuracy: 79.70%
total test samples: 1000
elapsed time: 2.46 sec
rate: 0.00 sec/sample


In [16]:
# grep retrieval engine
t = time()
cached_file = 'cached/grep_validation.csv'
pred = pd.DataFrame(columns=['no', 'key', 'prediction'])
correct, total = 0, 0
for index, row in val_set.iterrows():
    q = Question(row)
    grep.solve(q)
    pred = pred.append({'no': q.no, 'key': grep.key, 'prediction': grep.prediction}, ignore_index = True)
    if grep.prediction != Solver.UNSOLVED:
        if grep.prediction == q.ans: correct += 1
        total += 1
pred['no'] = pred['no'].astype(int)
pred.to_csv(cached_file, index=False)
print('validation accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total validation samples: {}'.format(total))

temp = total
cached_file = 'cached/grep_test.csv'
pred = pd.DataFrame(columns=['no', 'key', 'prediction'])
correct, total = 0, 0
for index, row in test_set.iterrows():
    q = Question(row)
    grep.solve(q)
    pred = pred.append({'no': q.no, 'key': grep.key, 'prediction': grep.prediction}, ignore_index = True)
    if grep.prediction != Solver.UNSOLVED:
        if grep.prediction == q.ans: correct += 1
        total += 1
elapsed = time() - t
pred['no'] = pred['no'].astype(int)
pred.to_csv(cached_file, index=False)

print('test accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total test samples: {}'.format(total))
print('elapsed time: {:.2f} sec'.format(elapsed))
print('rate: {:.2f} sec/sample'.format(float(elapsed) / float(temp + total)))

# validation accuracy: 93.77%
# total validation samples: 546
# test accuracy: 71.40%
# total test samples: 1000
# elapsed time: 482.07 sec
# rate: 0.31 sec/sample

validation accuracy: 93.77%
total validation samples: 546
test accuracy: 71.40%
total test samples: 1000
elapsed time: 4811.74 sec
rate: 3.11 sec/sample
